In [1]:
import torch
import sys
sys.path.append("../src")
sys.path.append("../src/models")
from train import getModel, prepare_dataset
from experiments import experiments
import argparse
import tqdm
import numpy as np
from datasets.ConcatDataset import ConcatDataset
import pandas as pd
import geopandas as gpd
import sklearn.metrics
import os
import matplotlib.pyplot as plt
import matplotlib.patheffects as path_effects

figdir="/home/marc/projects/ISPRS_CropTypeMapping/images/regions"

### Evaluate pretrained model with one dataloader

In [2]:
def evaluate(model, dataloader):

    probaslist = list()
    targetslist = list()
    idslist = list()
    for iteration, data in tqdm.tqdm(enumerate(dataloader)):
        inputs, targets, ids = data
        if torch.cuda.is_available():
            inputs = inputs.cuda()
            targets = targets.cuda()
            model.cuda()
        else:
            model.cpu()

        logprobabilities, deltas, pts, budget = model.forward(inputs.transpose(1, 2))
        probaslist.append(logprobabilities.exp().cpu().detach().numpy())
        targetslist.append(targets.cpu().detach().numpy())
        idslist.append(ids.numpy())

    ids = np.hstack(idslist)
    probas = np.vstack(probaslist)
    targets = np.vstack(targetslist)[:,0]

    return ids, targets, probas


### Get pretrained model and run it on all three regions

In [3]:

def evaluate_regions(experiment, seed=0, metric=sklearn.metrics.cohen_kappa_score, name="regions"):

    args = argparse.Namespace(experiment=experiment, seed=seed, batchsize=256, workers=0, hparamset=None)
    args = experiments(args)
    
    args.testregions = args.trainregions = ["holl"]
    holltraindataloader, holltestdataloader = prepare_dataset(args)

    args.testregions = args.trainregions = ["nowa"]
    nowatraindataloader, nowatestdataloader = prepare_dataset(args)

    args.testregions = args.trainregions = ["krum"]
    krumtraindataloader, krumtestdataloader = prepare_dataset(args)

    args.nclasses = holltraindataloader.dataset.nclasses
    classname = holltraindataloader.dataset.classname
    klassenname = holltraindataloader.dataset.klassenname
    args.seqlength = holltraindataloader.dataset.sequencelength
    args.input_dims = holltraindataloader.dataset.ndims
    
    model = getModel(args)
    model.load(f"/data/isprs/{name}/{seed}/{experiment}/model.pth")

    ids_holl, targets_holl, probas_holl = evaluate(model, holltestdataloader)
    ids_nowa, targets_nowa, probas_nowa = evaluate(model, nowatestdataloader)
    ids_krum, targets_krum, probas_krum = evaluate(model, krumtestdataloader)

    holl = metric(targets_holl, probas_holl.argmax(1))
    nowa = metric(targets_nowa, probas_nowa.argmax(1))
    krum = metric(targets_krum, probas_krum.argmax(1))
    
    return holl, nowa, krum

def plot(matrix, std_matrix=None):
    fig,ax = plt.subplots()
    ax.imshow(matrix)
    ax.set_xticks([0,1,2])
    ax.set_xticklabels(["Hollfeld", "Bavarian Forest", "Krumbach"])
    ax.set_yticks([0,1,2])
    ax.set_yticklabels(["Hollfeld", "Bavarian Forest", "Krumbach"])
    ax.set_ylabel("trained")
    ax.set_xlabel("evaluated")

    for i in np.arange(matrix.shape[0]):
        for j in np.arange(matrix.shape[0]):
            entry=f"{matrix[j,i]:.2f}"
            if std_matrix is not None:
                entry+="$^{" + f"\pm{std_matrix[j,i]:.2f}" + "}$"
            
            txt = ax.text(i,j,entry, ha="center", va="center", color="black",fontsize=14)
            txt.set_path_effects([path_effects.Stroke(linewidth=2, foreground='white'),
                           path_effects.Normal()])
            
    return fig


def save_plot(fig,fname):
    print(f"writing {fname}")
    fig.savefig(fname,bbox_inches='tight',transparent=True, density=300)
    
def get_matrix(seed, run_experiments, metric):

    hh, hn, hk = evaluate_regions(run_experiments[0], seed=seed, metric=metric)
    nh, nn, nk = evaluate_regions(run_experiments[1], seed=seed, metric=metric)
    kh, kn, kk = evaluate_regions(run_experiments[2], seed=seed, metric=metric)

    return np.array([[hh, hn, hk],[nh, nn, nk],[kh, kn, kk]])
    
def print_tex(m_mean, m_std, prefix=["","",""]):
    table=""
    for row_mean, row_std, pre in zip(m_mean,m_std, prefix):
        row = list()
        for mean, std in zip(row_mean,row_std):
            row.append(f"{mean:.2f}" + "$^{\pm"+f"{std:.2f}"+"}$")

        print(pre + " & ".join(row) + r" \\")
        
def get_matrix_all_seeds(run_experiments, metric):

    M = list()
    for seed in [0,1,2]:
        M.append(get_matrix(seed,run_experiments,metric))

    M = np.stack(M)
    
    return M



## Accuracy on preprocessed datasets

In [ ]:
run_experiments = ["isprs_gafholl_transformer","isprs_gafnowa_transformer","isprs_gafkrum_transformer"]
metric = sklearn.metrics.accuracy_score

original = sys.stdout
with open(os.devnull, 'w') as f:
    sys.stdout = f
    M = get_matrix_all_seeds(run_experiments, metric)
sys.stdout = original

m_mean = M.mean(0)
m_std = M.std(0)

print_tex(m_mean, m_std, prefix=["& Hollfeld & ", "trained & Bavarian Forest & ", "& Krumbach & "])
save_plot(plot(m_mean, m_std), os.path.join(figdir,"gaf_accuracy.png"))

## Kappa on preprocessed datasets

In [ ]:
run_experiments = ["isprs_gafholl_transformer","isprs_gafnowa_transformer","isprs_gafkrum_transformer"]
metric = sklearn.metrics.cohen_kappa_score

original = sys.stdout
with open(os.devnull, 'w') as f:
    sys.stdout = original
    M = get_matrix_all_seeds(run_experiments, metric)
sys.stdout = original

cm_mean = M.mean(0)
m_std = M.std(0)

print_tex(m_mean, m_std, prefix=["& Hollfeld & ", "trained & Bavarian Forest & ", "& Krumbach & "])
save_plot(plot(m_mean, m_std), os.path.join(figdir,"gaf_kappa.png"))

## Kappa on raw datasets

In [ ]:
run_experiments = ["isprs_tumholl_transformer","isprs_tumnowa_transformer","isprs_tumkrum_transformer"]
metric = sklearn.metrics.cohen_kappa_score

original = sys.stdout
with open(os.devnull, 'w') as f:
    sys.stdout = f
    M = get_matrix_all_seeds(run_experiments, metric)
sys.stdout = original

m_mean = M.mean(0)
m_std = M.std(0)

print_tex(m_mean, m_std, prefix=["& Hollfeld & ", "trained & Bavarian Forest & ", "& Krumbach & "])
save_plot(plot(m_mean, m_std), os.path.join(figdir,"tum_kappa.png"))

## Accuracy on raw datasets

In [ ]:
run_experiments = ["isprs_tumholl_transformer","isprs_tumnowa_transformer","isprs_tumkrum_transformer"]
metric = sklearn.metrics.accuracy_score

original = sys.stdout
with open(os.devnull, 'w') as f:
    sys.stdout = f
    M = get_matrix_all_seeds(run_experiments, metric)
sys.stdout = original

m_mean = M.mean(0)
m_std = M.std(0)

print_tex(m_mean, m_std, prefix=["& Hollfeld & ", "trained & Bavarian Forest & ", "& Krumbach & "])
save_plot(plot(m_mean, m_std), os.path.join(figdir,"tum_accuracy.png"))

## Accuracy on preprocessed datasets random split

In [ ]:
run_experiments = ["isprs_gafholl_transformer_randomsplit","isprs_gafnowa_transformer_randomsplit",
                   "isprs_gafkrum_transformer_randomsplit"]
metric = sklearn.metrics.accuracy_score

original = sys.stdout
with open(os.devnull, 'w') as f:
    sys.stdout = f
    M = get_matrix_all_seeds(run_experiments, metric)
sys.stdout = original

m_mean = M.mean(0)
m_std = M.std(0)

print_tex(m_mean, m_std, prefix=["& Hollfeld & ", "trained & Bavarian Forest & ", "& Krumbach & "])
save_plot(plot(m_mean, m_std), os.path.join(figdir,"gaf_accuracy_random.png"))

## Kappa on preprocessed datasets random split

In [ ]:
run_experiments = ["isprs_gafholl_transformer_randomsplit","isprs_gafnowa_transformer_randomsplit",
                   "isprs_gafkrum_transformer_randomsplit"]
metric = sklearn.metrics.cohen_kappa_score

original = sys.stdout
with open(os.devnull, 'w') as f:
    sys.stdout = f
    M = get_matrix_all_seeds(run_experiments, metric)
sys.stdout = original

m_mean = M.mean(0)
m_std = M.std(0)

print_tex(m_mean, m_std, prefix=["& Hollfeld & ", "trained & Bavarian Forest & ", "& Krumbach & "])
save_plot(plot(m_mean, m_std), os.path.join(figdir,"gaf_kappa_random.png"))

## Accuracy on raw datasets random split

In [ ]:
run_experiments = ["isprs_tumholl_transformer_randomsplit","isprs_tumnowa_transformer_randomsplit",
                   "isprs_tumkrum_transformer_randomsplit"]
metric = sklearn.metrics.accuracy_score

original = sys.stdout
with open(os.devnull, 'w') as f:
    sys.stdout = f
    M = get_matrix_all_seeds(run_experiments, metric)
sys.stdout = original

m_mean = M.mean(0)
m_std = M.std(0)

print_tex(m_mean, m_std, prefix=["& Hollfeld & ", "trained & Bavarian Forest & ", "& Krumbach & "])
save_plot(plot(m_mean, m_std), os.path.join(figdir,"tum_accuracy_random.png"))

## Kappa on raw datasets random split

In [ ]:
run_experiments = ["isprs_tumholl_transformer_randomsplit","isprs_tumnowa_transformer_randomsplit",
                   "isprs_tumkrum_transformer_randomsplit"]
metric = sklearn.metrics.cohen_kappa_score

original = sys.stdout
with open(os.devnull, 'w') as f:
    sys.stdout = f
    M = get_matrix_all_seeds(run_experiments, metric)
sys.stdout = original

m_mean = M.mean(0)
m_std = M.std(0)

print_tex(m_mean, m_std, prefix=["& Hollfeld & ", "trained & Bavarian Forest & ", "& Krumbach & "])
save_plot(plot(m_mean, m_std), os.path.join(figdir,"tum_kappa_random.png"))

Exception in thread IPythonHistorySavingThread:
Traceback (most recent call last):
  File "/home/marc/miniconda3/envs/pytorch/lib/python3.7/site-packages/IPython/core/history.py", line 780, in writeout_cache
    self._writeout_input_cache(conn)
  File "/home/marc/miniconda3/envs/pytorch/lib/python3.7/site-packages/IPython/core/history.py", line 764, in _writeout_input_cache
    (self.session_number,)+line)
sqlite3.IntegrityError: UNIQUE constraint failed: history.session, history.line

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/marc/miniconda3/envs/pytorch/lib/python3.7/site-packages/IPython/core/history.py", line 834, in run
    self.history_manager.writeout_cache(self.db)
  File "</home/marc/miniconda3/envs/pytorch/lib/python3.7/site-packages/decorator.py:decorator-gen-23>", line 2, in writeout_cache
  File "/home/marc/miniconda3/envs/pytorch/lib/python3.7/site-packages/IPython/core/history.py", line 58, in ne

## Train on holl

In [7]:
run_experiments = ["isprs_gaf_transformer"]
metric = sklearn.metrics.cohen_kappa_score

original = sys.stdout
#with open(os.devnull, 'w') as f:
#    sys.stdout = f
m = evaluate_regions("isprs_tum_transformer", seed=0, metric=metric, name="preraw")
#sys.stdout = original

m_mean = M.mean(0)
m_std = M.std(0)

#print_tex(m_mean, m_std, prefix=["& Hollfeld & ", "trained & Bavarian Forest & ", "& Krumbach & "])
#save_plot(plot(m_mean, m_std), os.path.join(figdir,"tum_kappa_random.png"))

Initializing BavarianCropsDataset test partition in holl
read 12 classes
precached dataset files found at /data/BavarianCrops/npy/classmapping.isprs2.csv/blocks/holl/test
loaded 9583 samples
Dataset /data/BavarianCrops. region holl. partition test. X:9583x(144, 13), y:(9583,) with 12 classes
Initializing BavarianCropsDataset trainvalid partition in holl
read 12 classes
precached dataset files found at /data/BavarianCrops/npy/classmapping.isprs2.csv/blocks/holl/trainvalid
loaded 24767 samples
Dataset /data/BavarianCrops. region holl. partition trainvalid. X:24767x(71, 13), y:(24767,) with 12 classes
setting random seed to 0
Initializing BavarianCropsDataset test partition in nowa
read 12 classes
precached dataset files found at /data/BavarianCrops/npy/classmapping.isprs2.csv/blocks/nowa/test
loaded 3547 samples
Dataset /data/BavarianCrops. region nowa. partition test. X:3547x(287, 13), y:(3547,) with 12 classes
Initializing BavarianCropsDataset trainvalid partition in nowa
read 12 class

RuntimeError: Error(s) in loading state_dict for TransformerEncoder:
	Unexpected key(s) in state_dict: "encoder.layer_stack.3.slf_attn.w_qs.weight", "encoder.layer_stack.3.slf_attn.w_qs.bias", "encoder.layer_stack.3.slf_attn.w_ks.weight", "encoder.layer_stack.3.slf_attn.w_ks.bias", "encoder.layer_stack.3.slf_attn.w_vs.weight", "encoder.layer_stack.3.slf_attn.w_vs.bias", "encoder.layer_stack.3.slf_attn.layer_norm.weight", "encoder.layer_stack.3.slf_attn.layer_norm.bias", "encoder.layer_stack.3.slf_attn.fc.weight", "encoder.layer_stack.3.slf_attn.fc.bias", "encoder.layer_stack.3.pos_ffn.w_1.weight", "encoder.layer_stack.3.pos_ffn.w_1.bias", "encoder.layer_stack.3.pos_ffn.w_2.weight", "encoder.layer_stack.3.pos_ffn.w_2.bias", "encoder.layer_stack.3.pos_ffn.layer_norm.weight", "encoder.layer_stack.3.pos_ffn.layer_norm.bias", "encoder.layer_stack.4.slf_attn.w_qs.weight", "encoder.layer_stack.4.slf_attn.w_qs.bias", "encoder.layer_stack.4.slf_attn.w_ks.weight", "encoder.layer_stack.4.slf_attn